<a href="https://colab.research.google.com/github/OjashKush/UR-FALL-GS/blob/main/fall_gs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.6 MB/s eta 0:00:00


In [5]:
import os
import urllib.request
import zipfile
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from PIL import Image
import time
import pickle

import os
import urllib.request
import zipfile
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from PIL import Image

# Define the base URL and local directory
base_url = "http://fenix.ur.edu.pl/~mkepski/ds/data/"
local_dir = "/content/urfall_dataset"


def download_and_extract(file_name):
    """Download and extract a zip file."""
    url = base_url + file_name
    zip_path = os.path.join(local_dir, file_name)

    # Download the file
    print(f"Downloading {file_name}...")
    urllib.request.urlretrieve(url, zip_path)

    # Extract the file
    print(f"Extracting {file_name}...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(local_dir)

    # Remove the zip file
    os.remove(zip_path)

def prepare_dataset():
    """Prepare the UR Fall dataset."""

    # Create the local directory if it doesn't exist
    os.makedirs(local_dir, exist_ok=True)

    # Download and extract depth data for falls
    for i in range(1, 31):
        file_name = f"fall-{i:02d}-cam0-d.zip"
        download_and_extract(file_name)

    # Download and extract depth data for ADLs
    for i in range(1, 41):
        file_name = f"adl-{i:02d}-cam0-d.zip"
        download_and_extract(file_name)

def process_data():
    """Process the downloaded data and create CSV files."""
    fall_data = []
    adl_data = []

    # Process fall data
    for i in range(1, 31):
        folder = os.path.join(local_dir, f"fall-{i:02d}-cam0-d")
        if os.path.exists(folder):
            for file in os.listdir(folder):
                if file.endswith('.png'):
                    fall_data.append({'file': os.path.join(folder, file), 'label': 1})

    # Process ADL data
    for i in range(1, 41):
        folder = os.path.join(local_dir, f"adl-{i:02d}-cam0-d")
        if os.path.exists(folder):
            for file in os.listdir(folder):
                if file.endswith('.png'):
                    adl_data.append({'file': os.path.join(folder, file), 'label': 0})

    # Combine and shuffle data
    all_data = fall_data + adl_data
    np.random.shuffle(all_data)

    # Create DataFrame
    df = pd.DataFrame(all_data)

    return df

def split_dataset(df, test_size=0.2, random_state=42):
    """Split the dataset into train and test sets."""

    # Split the data
    train_df, test_df = train_test_split(df, test_size=test_size, random_state=random_state, stratify=df['label'])

    # Save the split datasets
    train_df.to_csv(os.path.join(local_dir, "train_data.csv"), index=False)
    test_df.to_csv(os.path.join(local_dir, "test_data.csv"), index=False)

    print(f"Train set size: {len(train_df)}")
    print(f"Test set size: {len(test_df)}")

    return train_df, test_df

class URFallDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        file_path = self.dataframe.iloc[idx]['file']
        label = self.dataframe.iloc[idx]['label']

        # Load image using PIL
        image = Image.open(file_path).convert('L')  # Convert to grayscale
        if self.transform:
            image = self.transform(image)

        return image, label

# Data transformations
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to a fixed size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

class MotionModule(nn.Module):
    def __init__(self):
        super(MotionModule, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=64, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        return x

class GSTCAN(nn.Module):
    def __init__(self, num_classes=2):
        super(GSTCAN, self).__init__()
        self.motion_module = MotionModule()
        self.gcn_layer = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=1)
        self.tcn_layer = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        self.cam_layer = nn.Sequential(
            nn.Conv1d(256, 256, kernel_size=1),
            nn.Sigmoid()
        )
        self.gap = nn.AdaptiveAvgPool2d(1)
        self.classifier = nn.Linear(256, num_classes)

    def forward(self, x):
        motion_features = self.motion_module(x)
        gcn_output = self.gcn_layer(motion_features)
        tcn_input = gcn_output.view(gcn_output.size(0), gcn_output.size(1), -1)
        tcn_output = self.tcn_layer(tcn_input)
        cam_output = self.cam_layer(tcn_output)
        pooled_features = self.gap(cam_output.unsqueeze(-1))
        output = self.classifier(pooled_features.view(pooled_features.size(0), -1))
        return output

def save_checkpoint(model, optimizer, epoch, train_loss, test_accuracy, filename):
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'train_loss': train_loss,
        'test_accuracy': test_accuracy
    }
    torch.save(checkpoint, filename)

def load_checkpoint(model, optimizer, filename):
    checkpoint = torch.load(filename)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    return checkpoint['epoch'], checkpoint['train_loss'], checkpoint['test_accuracy']

def save_transforms(transforms, filename):
    with open(filename, 'wb') as f:
        pickle.dump(transforms, f)

def load_transforms(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

def train_model(model, train_loader, test_loader, num_epochs=10, checkpoint_dir='checkpoints'):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    os.makedirs(checkpoint_dir, exist_ok=True)

    start_epoch = 0
    best_accuracy = 0.0

    # Check if there's a checkpoint to resume from
    checkpoints = [f for f in os.listdir(checkpoint_dir) if f.startswith('checkpoint_epoch_')]
    if checkpoints:
        latest_checkpoint = max(checkpoints, key=lambda x: int(x.split('_')[2].split('.')[0]))
        checkpoint_path = os.path.join(checkpoint_dir, latest_checkpoint)
        start_epoch, train_loss, best_accuracy = load_checkpoint(model, optimizer, checkpoint_path)
        print(f"Resuming from epoch {start_epoch} with best accuracy: {best_accuracy:.2f}%")
        start_epoch += 1  # Start from the next epoch

    total_start_time = time.time()

    for epoch in range(start_epoch, num_epochs):
        model.train()
        epoch_start_time = time.time()
        running_loss = 0.0

        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            if batch_idx % 10 == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

        epoch_loss = running_loss / len(train_loader)

        # Testing loop
        model.eval()
        correct = 0
        total = 0

        with torch.no_grad():
            for data, target in test_loader:
                data, target = data.to(device), target.to(device)
                output = model(data)
                _, predicted = torch.max(output.data, 1)
                total += target.size(0)
                correct += (predicted == target).sum().item()

        accuracy = 100 * correct / total
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Test accuracy: {accuracy:.2f}%')

        # Save checkpoint
        checkpoint_path = os.path.join(checkpoint_dir, f'checkpoint_epoch_{epoch+1}.pth')
        save_checkpoint(model, optimizer, epoch+1, epoch_loss, accuracy, checkpoint_path)

        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_model_path = os.path.join(checkpoint_dir, 'best_model.pth')
            torch.save(model.state_dict(), best_model_path)

        epoch_end_time = time.time()
        epoch_duration = epoch_end_time - epoch_start_time
        estimated_time_left = epoch_duration * (num_epochs - epoch - 1)

        print(f'Epoch duration: {epoch_duration:.2f} seconds')
        print(f'Estimated time left: {estimated_time_left/60:.2f} minutes')

    total_end_time = time.time()
    total_duration = total_end_time - total_start_time
    print(f'Total training time: {total_duration/60:.2f} minutes')


if __name__ == "__main__":
    prepare_dataset()
    df = process_data()
    train_df, test_df = split_dataset(df)

    # Create and save the data transforms
    data_transforms = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5])
    ])
    save_transforms(data_transforms, 'data_transforms.pkl')

    train_dataset = URFallDataset(train_df, transform=data_transforms)
    test_dataset = URFallDataset(test_df, transform=data_transforms)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    model = GSTCAN(num_classes=2)
    train_model(model, train_loader, test_loader)

    print("Dataset preparation and model training completed.")

Extracting fall-01-cam0-d.zip...
Extracting fall-02-cam0-d.zip...
Extracting fall-03-cam0-d.zip...
Extracting fall-04-cam0-d.zip...
Extracting fall-05-cam0-d.zip...
Extracting fall-06-cam0-d.zip...
Extracting fall-07-cam0-d.zip...
Extracting fall-08-cam0-d.zip...
Extracting fall-09-cam0-d.zip...
Extracting fall-10-cam0-d.zip...
Extracting fall-11-cam0-d.zip...
Extracting fall-12-cam0-d.zip...
Extracting fall-13-cam0-d.zip...
Extracting fall-14-cam0-d.zip...
Extracting fall-15-cam0-d.zip...
Extracting fall-16-cam0-d.zip...
Extracting fall-17-cam0-d.zip...
Extracting fall-18-cam0-d.zip...
Extracting fall-19-cam0-d.zip...
Extracting fall-20-cam0-d.zip...
Extracting fall-21-cam0-d.zip...
Extracting fall-22-cam0-d.zip...
Extracting fall-23-cam0-d.zip...
Extracting fall-24-cam0-d.zip...
Extracting fall-25-cam0-d.zip...
Extracting fall-26-cam0-d.zip...
Extracting fall-27-cam0-d.zip...
Extracting fall-28-cam0-d.zip...
Extracting fall-29-cam0-d.zip...
Extracting fall-30-cam0-d.zip...
Extracting